In [9]:
import numpy as np
import string
np.random.seed(1234)

In [10]:
initial = {}
first_order = {}
second_order = {}

In [11]:
def remove_punctuation(s):
  return s.translate(str.maketrans('','',string.punctuation))

In [12]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost\.txt

File ‘robert_frost.txt’ already there; not retrieving.



In [13]:
def add2dict(d, k, v):
  if k not in d:
    d[k] = []
  d[k].append(v)

In [15]:
for line in open('robert_frost.txt'):
  tokens = remove_punctuation(line.rstrip().lower()).split()
  T = len(tokens)
  for i in range(T):
    t = tokens[i]
    if i == 0:
      # measure the distribution of the first word
      initial[t] = initial.get(t, 0.) + 1
    else:
      t_1 = tokens[i-1]
      if i == T - 1:
        # measure the distribution of the last word
        add2dict(second_order, (t_1, t), 'END')
      if i == 1:
        # measure the distribution of the second word, given only the first word
        add2dict(first_order, t_1, t)
      else:
        t_2 = tokens[i-2]
        add2dict(second_order, (t_2, t_1), t)

In [16]:
# normalize the distribution
initial_total = sum(initial.values())
for t, c in initial.items():
  initial[t] = c / initial_total

In [17]:
# convert [cat, cat, dog.....] into {cat: 0.2, dog:0.5 etc}
def list2dict(ts):
  # turn each list of probabilities into a dictionary of probabilities
  d = {}
  n = len(ts)
  for t in ts:
    d[t] = d.get(t, 0.) + 1
  for t, c in d.items():
    d[t] = c / n
  return d


  for t_1, ts in first_order.items():
    # replace list with dictionary of probabilities
    first_order[t_1] = list2dict(ts)
  for k, ts in second_order.items():
    second_order[k] = list2dict(ts)

In [22]:
def sample_word(d):
  # print("d: ", d)
  p0 = np.random.random()
  # print("p0: ", p0)
  cummulative = 0
  for t, p in d.items():
    cummulative += p
    if p0 < cummulative:
      return t
  assert(False) # should never reach here

In [23]:
def generate():
  for i in range(4): #generate only for lines
    sentence = []
    # initial word
    w0 = sample_word(initial)
    sentence.append(w0)
    # sample second word
    w1 = sample_word(first_order[w0])
    sentence.append(w1)
    # second order transitions until END
    while True:
      w2 = sample_word(second_order[(w0, w1)])
      if w2 == 'END':
        break
      sentence.append(w2)
      w0 = w1
      w1 = w2
    print(' '.join(sentence))

In [29]:
# Convert lists in first_order to dictionaries of probabilities
for t_1, ts in list(first_order.items()):
  first_order[t_1] = list2dict(ts)

# Convert lists in second_order to dictionaries of probabilities
for k, ts in list(second_order.items()):
  second_order[k] = list2dict(ts)

generate()

who wants to hear the voices seem to hear today
the swarm
it has not ranged
from my advantage on a plane of snow
